In [ ]:
import os
import matplotlib.pyplot as plt
import h5py
import numpy as np
import yaml

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 20,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directories ----------------------
mqlib_results_dir_path = f"{script_dir}/outputs/qbp/very_large/2024-05-16/13-15-25"
annealing_results_dirs = f"{script_dir}/outputs/qbp/very_large/"
# ---------------------------------------------------------------------------
simcim_results_path = f"{mqlib_results_dir_path}/simcim_result"
mqlib_results_path = f"{mqlib_results_dir_path}/mqlib_result"
annealing_results_dirs_list = os.walk(annealing_results_dirs).__next__()[1]

fig = plt.figure()
plt.plot()
total_evolution_times = []
annealing_energies = []
for annealing_results_dir in annealing_results_dirs_list:
    annealing_results_dir = f"{annealing_results_dirs}{annealing_results_dir}"
    subresults_dirs = os.walk(annealing_results_dir).__next__()[1]
    for subresults_dir in subresults_dirs:
        final_dir = f"{annealing_results_dir}/{subresults_dir}/result"
        config_path = f"{annealing_results_dir}/{subresults_dir}/.hydra/config.yaml"
        with open(config_path, "r") as f:
            raw_config = f.read()
            config = yaml.safe_load(raw_config)
            time_steps_num =int(config["quantum_annealing_schedule"]["steps_number"])
            time_step_size = float(config["quantum_annealing_schedule"]["total_time_step_size"])
            total_evolution_times.append(time_steps_num * time_step_size)
        with h5py.File(final_dir, "r") as f:
            annealing_energies.append(np.array(f["quantum_annealer_energy_wo_sampling"]))
time_indices = np.argsort(total_evolution_times)
total_evolution_times = np.array(total_evolution_times)[time_indices]
annealing_energies = np.array(annealing_energies)[time_indices]
with h5py.File(simcim_results_path, "r") as f:
    simcim_energy = np.array(f["simcim_energy"])
heuristics_energy = [simcim_energy]
heuristics_names = ["SIMCIM"]
with h5py.File(mqlib_results_path, "r") as f:
    for key in f.keys():
        if "energy" in key:
            energy = np.array(f[key])
            heuristics_energy.append(energy)
            heuristics_names.append(key.split("_")[0])
name_and_energy = list(zip(heuristics_names, heuristics_energy))
name_and_energy.sort(key=lambda x: x[1])
cmap = plt.get_cmap('winter')
cmap = [cmap(x) for x in np.linspace(0, 1, len(name_and_energy))]
min_heuristic_energy = name_and_energy[0][1]
for (name, energy), color in zip(name_and_energy, cmap):
    plt.axhline(energy - annealing_energies[-1], color=color)
plt.plot(total_evolution_times, annealing_energies - annealing_energies[-1], '-ko', label=r"${\rm Quantum \ Annealing}$")
plt.ylabel(r"$\Delta E$")
plt.xlabel(r"${\rm Time}$")
plt.xscale("log")
plt.yscale("symlog")
plt.xticks([10, 100])
plt.legend(frameon=False)
print(list(map(lambda x: x[0], name_and_energy)))
print(f"Best annealing energy: {min(annealing_energies)}, best heuristic energy: {min_heuristic_energy}")

In [ ]:
fig.savefig(f"{mqlib_results_dir_path}/benchmark.pdf",  bbox_inches="tight")